In [13]:
# !pip install ogame

# https://github.com/alaingilbert/pyogame/wiki/Code-Snippets

import os
import sys
module_path = os.path.abspath(os.path.join('../pyogame'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ogame import OGame, constants
import time
import re
from bs4 import BeautifulSoup

import json

o = OGame(
    universe="Ymir",
    # universe='Volans',
    username="samchakerian@gmail.com",
    password="$R3e@W1q1q"
)


# to request esp reports
def BeautifulSoup4(response):
    parsed = BeautifulSoup(response, features="html5lib")

    def find_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find(**kwargs)

    def find_all_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find_all(**kwargs)

    parsed.find_partial = find_partial
    parsed.find_all_partial = find_all_partial
    return parsed


def get_reports():
    
    response = o.session.get(
        url=o.index_php,
        params={'page': 'messages',
                'tab': 20,
                'ajax': 1}
    ).text
    bs4 = BeautifulSoup4(response)
    report_links = [
        link['href']
        for link in bs4.find_all_partial(href='page=messages&messageId')
    ]
    
    data = []
    for link in report_links:
        response = o.session.get(link).text
        bs4 = BeautifulSoup4(response)
        data.append({'link': link, 'text': bs4})
        
    return data


def clean_reports(data):
    
    for report in data:
        rep_res = [_.text for _ in report['text'].find_all('span', class_="res_value")]
        rep_coord = report['text'].find('span', class_="msg_title new blue_txt").text
        try:
            rep_def = [_.text for _ in report['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]
        except:
            rep_def = None
        report['results'] = {
            'res': rep_res,
            'rep_coord': rep_coord,
            'rep_def': rep_def
        }
        
    return [_['results'] for _ in data]


### espionage nearby inactives ###

# Iterate thru galaxies
# Cut off after 10
# Check the reports, append data (cleaned)
# Continue with other 10, until done

data = []

r = 25
h = 447
s = h-r
e = h+r
if e > 499:
    e = 499
    
counter = 0
for galaxy in range(s, e):
    for planet in o.galaxy(constants.coordinates(1, galaxy)):
        
        # stop, check reports
        if counter >= 9:
            temp_data = get_reports()
            temp_data = clean_reports(temp_data)
            data.extend(temp_data)
            counter = 0
            time.sleep(5)
        
        # send the probes
        if 'inactive' in planet.status and 'vacation' not in planet.status:
            send = o.send_fleet(
                constants.mission.spy, 
                o.planet_ids()[0], 
                planet.position, 
                [constants.ships.espionage_probe(1)]
            )
            counter+=1
            print(galaxy, planet.status, planet.name, planet.rank, send)

422 ['inactive'] Homeworld 1245 True
423 ['inactive'] Homeworld 1312 True
424 ['inactive'] Homeworld 1593 True
425 ['inactive'] Homeworld 1350 True
425 ['inactive'] Homeworld 1780 True
430 ['inactive'] Homeworld 1302 True
431 ['inactive'] Ymirs Tears 1662 True
431 ['inactive'] Colony 821 True
431 ['inactive'] Alpha 821 True
431 ['inactive'] Homeworld 1557 True
432 ['inactive'] Homeworld 1231 True
432 ['inactive'] Homeworld 1626 True
433 ['inactive'] Homeworld 1343 True
433 ['inactive'] Homeworld 1507 True
434 ['inactive'] Homeworld 1558 True
434 ['inactive'] Homeworld 992 True
435 ['inactive'] Homeworld 1242 True
436 ['inactive'] Homeworld 1435 True
437 ['inactive'] Homeworld 1179 True
439 ['inactive'] Homeworld 1748 True
439 ['inactive'] Homeworld 1588 True
439 ['inactive'] Homeworld 1370 True
440 ['inactive'] Homeworld 1663 True
441 ['inactive'] Homeworld 1490 True
444 ['inactive'] Homeworld 1467 True
444 ['inactive'] Howak 1133 True
445 ['inactive'] Homeworld 1372 True
445 ['inactiv

In [5]:
# find how many res you can get:
coords_and_res = []
for _ in data:
    if _['rep_def'] == None:
        res_clean = [int(__.replace('.', '')) for __ in _['res'][:3]]
        coords_and_res.append([sum(res_clean), [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")]])

# backup
json.dump(sorted(coords_and_res, reverse=True), open('to_raid.json', 'w'))

In [6]:
# reread
coords_and_res = json.load(open('to_raid.json', 'r'))
sorted(coords_and_res, reverse=True)

[[127789, [1, 466, 12]],
 [82091, [1, 434, 12]],
 [30000, [1, 463, 10]],
 [25843, [1, 430, 10]],
 [24141, [1, 444, 12]],
 [21734, [1, 435, 8]],
 [21416, [1, 448, 10]],
 [20074, [1, 439, 4]],
 [20000, [1, 466, 8]],
 [20000, [1, 465, 4]],
 [20000, [1, 441, 10]],
 [20000, [1, 440, 8]],
 [20000, [1, 436, 12]],
 [20000, [1, 434, 8]],
 [20000, [1, 432, 12]],
 [20000, [1, 431, 12]],
 [20000, [1, 431, 8]],
 [19702, [1, 449, 12]],
 [18734, [1, 432, 8]],
 [18717, [1, 439, 12]],
 [16333, [1, 451, 12]],
 [15643, [1, 452, 6]],
 [15557, [1, 451, 6]],
 [15373, [1, 444, 8]],
 [14533, [1, 439, 6]],
 [14442, [1, 433, 6]],
 [13444, [1, 433, 8]],
 [11599, [1, 466, 10]],
 [8209, [1, 431, 9]],
 [8102, [1, 445, 4]],
 [8102, [1, 445, 4]]]

In [20]:
### send raids ###

# get ships
ships_ = o.ships(id)

# deploy ships
lcs = ships_.large_transporter.amount
scs = ships_.small_transporter.amount

# for _ in sorted(coords_and_res, reverse=True)[:]:

#     coords = _[1]
    
#     if lcs > 0:
#         if _[0]/2 > 25000:
#             n = int(_[0]/2/25000)
#         else:
#             n = 1 
#         to_send = constants.ships.large_transporter(n)
#         lcs -= n
    
#     elif lcs < 1 and scs > 0:
#         n = int(_[0]/2/5000)
#         to_send = constants.ships.small_transporter(n)
#         scs -= n
        
#     else:
#         break

#     send = o.send_fleet(
#         constants.mission.attack,
#         o.planet_ids()[0],
#         (coords[0], coords[1], coords[2], 1),
#         [to_send]
#     )
#     print(_, to_send, send)
#     print('remaining large:', lcs, 'remaining small:', scs)

In [19]:
def allocate_ships(coord_and_res, lcs, scs):
    
    coords = coord_and_res[1]
    
    if lcs > 0:
        if coord_and_res[0]/2 > 25000:
            n = int(coord_and_res[0]/2/25000)
            to_send = constants.ships.large_transporter(n)
            lcs -= n
            
        else:
            n = 1 
        to_send = constants.ships.large_transporter(n)
        lcs -= n
        res_dep = n*25000
    
    elif lcs < 1 and scs > 0:
        n = int(coord_and_res[0]/2/5000)
        to_send = constants.ships.small_transporter(n)
        scs -= n
        res_dep = n*5000
        
    else:
        return

    send = o.send_fleet(
        constants.mission.attack,
        o.planet_ids()[0],
        (coords[0], coords[1], coords[2], 1),
        [to_send]
    )
    print(coord_and_res, to_send, send)
    print('remaining large:', lcs, 'remaining small:', scs)
    
    res_remaining = coord_and_res[0] - res_dep
    
    return lcs, scs, res_remaining

In [17]:
### waveraiding ###

# get ships
ships_ = o.ships(id)

# deploy ships
lcs = ships_.large_transporter.amount
scs = ships_.small_transporter.amount

for _ in sorted(coords_and_res, reverse=True)[:]:

    lcs, scs, res_remain = allocate_ships(_, lcs, scs)
    if res_remain/2 > 25000:
        time.sleep(3)
        lcs, scs, res_remain = allocate_ships([res_remain, _[1]], lcs, scs)

[127789, [1, 466, 12]] (203, 2, 'shipyard') True
remaining large: 5 remaining small: 13
[77789, [1, 466, 12]] (203, 1, 'shipyard') True
remaining large: 3 remaining small: 13
[82091, [1, 434, 12]] (203, 1, 'shipyard') True
remaining large: 1 remaining small: 13
[57091, [1, 434, 12]] (203, 1, 'shipyard') True
remaining large: -1 remaining small: 13
[30000, [1, 463, 10]] (202, 3, 'shipyard') True
remaining large: -1 remaining small: 10
[25843, [1, 430, 10]] (202, 2, 'shipyard') True
remaining large: -1 remaining small: 8
[24141, [1, 444, 12]] (202, 2, 'shipyard') True
remaining large: -1 remaining small: 6
[21734, [1, 435, 8]] (202, 2, 'shipyard') True
remaining large: -1 remaining small: 4
[21416, [1, 448, 10]] (202, 2, 'shipyard') True
remaining large: -1 remaining small: 2
[20074, [1, 439, 4]] (202, 2, 'shipyard') True
remaining large: -1 remaining small: 0


TypeError: cannot unpack non-iterable NoneType object

In [21]:
res = o.research()
res.computer.level

7

In [19]:
o.build(constants.research.computer, id=o.planet_ids()[0])

In [8]:
o.relogin()

True

In [22]:
id = o.planet_ids()[0]
supply = o.supply(id)
res = o.resources(id)

# check bldg construction
print('construction:', {
    'sp': supply.solar_plant.in_construction,
    'mm': supply.metal_mine.in_construction,
    'cm': supply.crystal_mine.in_construction,
    'dm': supply.deuterium_mine.in_construction
})

# is possible
print('possible:', {
    'sp': supply.solar_plant.is_possible,
    'mm': supply.metal_mine.is_possible,
    'cm': supply.crystal_mine.is_possible,
    'dm': supply.deuterium_mine.is_possible
})

# check resources
print('resources:', {
    'sp': res.energy,
    'mm': res.metal,
    'cm': res.crystal,
    'dm': res.deuterium
})

# check levels
print('levels:', {
    'sp': supply.solar_plant.level,
    'mm': supply.metal_mine.level,
    'cm': supply.crystal_mine.level,
    'dm': supply.deuterium_mine.level
})

construction: {'sp': False, 'mm': False, 'cm': False, 'dm': False}
possible: {'sp': True, 'mm': True, 'cm': True, 'dm': True}
resources: {'sp': 194, 'mm': 188373, 'cm': 72153, 'dm': 73546}
levels: {'sp': 14, 'mm': 10, 'cm': 11, 'dm': 9}
